<a href="https://colab.research.google.com/github/shailja-thakur/benchmarking-LLM-Verilog/blob/main/VGen_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install requirements

In [ ]:

!pip install git+https://github.com/huggingface/transformers.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-2rxx1t4t
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-2rxx1t4t
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 14.9 MB/s 
     |████████████████████████████████| 163 kB 69.6 MB/s 
  Created wheel for transformers: filename=transformers-4.25.0.dev0-py3-none-any.whl size=5700835 sha256=2bc1ef10e63590cf617bf7a821b41d4311bb8ac7ed34d8596a789aa62a267ccb
  Stored in directory: /tmp/pip-ephem-wheel-cache-uvfbddun/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov 15 21:51:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


# Load model and tokenizer

In [ ]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")


model_name = "shailja/CodeGen_2B_Verilog"

tokenizer = AutoTokenizer.from_pretrained("shailja/CodeGen_2B_Verilog")

model = AutoModelForCausalLM.from_pretrained("shailja/CodeGen_2B_Verilog").to(device)



Using GPU


# Try out the model for sampling Verilog

In [ ]:

prompt = "//module half adder "
# prompt="// Design a 2-to-1 multiplexer.\n module mux( \n    input [4:0] a, b,\n    input sel,\n    output [4:0] out );\n // When sel=0, assign a to out. \n// When sel=1, assign b to out."
n_steps = 1
n = 5
end_pattern='endmodule'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
# Sample
sample = model.generate(input_ids, max_length=128, temperature=0.5, top_p=0.9)
print(tokenizer.decode(sample[0], truncate_before_pattern=[r"endmodule"]) + end_pattern)


# Try out the model for sampling Verilog per token basis

In [ ]:

iteration = dict()
with torch.no_grad():
    while True:
        
        iteration["Input"] = input_ids
        
        output = model(input_ids)

        # select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # print(sorted_ids)
        
        
        # store tokens with highest probability
        for choice_idx in range(n):
            token_id = sorted_ids[choice_idx]
            
            token_prob = next_token_probs[token_id].cpu().numpy()
            # print(token_id, token_prob)
            # print(tokenizer.decode(token_id.view(1,-1)))
            
            token_choice = (              
                f"{tokenizer.decode(token_id)} ({100*token_prob:.2f}%)"
            )
            
            iteration[f"{choice_idx+1}"] = token_choice

        #Append predictions to list
        input_ids = torch.cat([input_ids, sorted_ids[None,0, None]], dim=-1)
        
        # condition checks on stop_words detected
        if 'endmodule' in tokenizer.batch_decode(input_ids)[0]: break
        
print(tokenizer.batch_decode(input_ids)[0])